In [155]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd

In [156]:
load_dotenv()
db_pass=os.getenv("DB_PASS")

In [157]:

def create_database(db_pass):
    # connect to default database
    try:
        initial_conn = psycopg2.connect(dbname="postgres", user="postgres",
                                        host="localhost", port="5432", password=db_pass)
        initial_conn.autocommit = True
        print("connection successful")
    except Exception as error:
        print(f"connection failed :{error}")

    # create twitter database
    db_name = "twitter"
    with initial_conn.cursor() as cur:
        cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{db_name}'")
        exists = cur.fetchone()
        if not exists:
            cur.execute(f""" CREATE DATABASE {db_name}""")
            print(f"Database {db_name} created ")
        else:
            print(f"Database {db_name} already exists ")
    #close connection to default database
    initial_conn.close()

    # connect to twitter database
    try:
        conn = psycopg2.connect(dbname=db_name, user="postgres",
                                        host="localhost", port="5432", password=db_pass)
        conn.autocommit = True
        cur = conn.cursor()
        print(f"connection successful to {db_name} database")
    except Exception as error:
        print(f"connection failed :{error}")

    return conn


### Preparing the dataset

In [158]:
df = pd.read_csv("../twitter_data/tweets.csv")
df.head()

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655


In [159]:
df.isnull().sum()

author                  0
content                 0
country             52506
date_time               0
id                      0
language                0
latitude            52541
longitude           52541
number_of_likes         0
number_of_shares        0
dtype: int64

In [160]:
df.drop(columns=["latitude", "longitude","country"],axis=1,inplace=True)
df

,author,content,date_time,id,language,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,12/01/2017 19:52,8.196330e+17,en,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,11/01/2017 08:38,8.191010e+17,en,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,11/01/2017 02:52,8.190140e+17,en,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,11/01/2017 02:44,8.190120e+17,en,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,10/01/2017 05:22,8.186890e+17,en,17620,4655
...,...,...,...,...,...,...,...
52537,ddlovato,Life couldn't be better right now. 😊,06/01/2015 23:10,5.526030e+17,en,32799,23796
52538,ddlovato,First Monday back in action. I'd say 21.6 mile...,06/01/2015 02:17,5.522880e+17,en,21709,12511
52539,ddlovato,"Crime shows, buddy, snuggles = the perfect Sun...",05/01/2015 03:42,5.519470e+17,en,25269,15583
52540,ddlovato,❄️ http://t.co/sHCFdPpGPa,05/01/2015 00:06,5.518920e+17,und,15985,10456


In [161]:
df['date_time'] = pd.to_datetime(
    df['date_time'], format='%d/%m/%Y %H:%M', errors='coerce')
# df = df.dropna(subset=['date_time'])

In [162]:
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52542 entries, 0 to 52541
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   author            52542 non-null  string        
 1   content           52542 non-null  string        
 2   date_time         52542 non-null  datetime64[ns]
 3   id                52542 non-null  Int64         
 4   language          52542 non-null  string        
 5   number_of_likes   52542 non-null  Int64         
 6   number_of_shares  52542 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1), string(3)
memory usage: 3.0 MB


In [163]:
df.rename(columns={'date_time':'tweeted_time','id':'tweet_id'},inplace = True)


In [164]:
df.drop_duplicates(subset=['tweet_id'], inplace=True)
df

,author,content,tweeted_time,tweet_id,language,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,2017-01-12 19:52:00,819633000000000000,en,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,2017-01-11 08:38:00,819101000000000000,en,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,2017-01-11 02:52:00,819014000000000000,en,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,2017-01-11 02:44:00,819012000000000000,en,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,2017-01-10 05:22:00,818689000000000000,en,17620,4655
...,...,...,...,...,...,...,...
52537,ddlovato,Life couldn't be better right now. 😊,2015-01-06 23:10:00,552603000000000000,en,32799,23796
52538,ddlovato,First Monday back in action. I'd say 21.6 mile...,2015-01-06 02:17:00,552288000000000000,en,21709,12511
52539,ddlovato,"Crime shows, buddy, snuggles = the perfect Sun...",2015-01-05 03:42:00,551947000000000000,en,25269,15583
52540,ddlovato,❄️ http://t.co/sHCFdPpGPa,2015-01-05 00:06:00,551892000000000000,und,15985,10456


## Data Model Created

![Data Model](Data_Model.png)

## Create tables according the data model

### Create schema

In [165]:
conn = create_database(db_pass)
with conn.cursor() as cur:
    cur.execute(""" CREATE SCHEMA IF NOT EXISTS twitter_data""")
    print(f"Schema twitter_data created ")

connection successful
Database twitter created 
connection successful to twitter database
Schema twitter_data created 


#### Create Author Table

In [166]:
with conn.cursor() as cur:
    cur.execute(""" CREATE TABLE IF NOT EXISTS twitter_data.Accounts(
                author VARCHAR PRIMARY KEY)""")
    print("Table Accounts Created")

Table Accounts Created


#### Create language table

In [167]:
with conn.cursor() as cur:
    cur.execute(""" CREATE TABLE IF NOT EXISTS twitter_data.Language(
                language VARCHAR PRIMARY KEY)""")
    print("Table Language Created")

Table Language Created



#### Create Tweets Table

In [168]:
with conn.cursor() as cur:
    cur.execute(""" CREATE TABLE IF NOT EXISTS twitter_data.Tweets(
                tweet_id BIGINT PRIMARY KEY,
                author VARCHAR ,
                content TEXT,
                tweeted_time TIMESTAMP,
                language TEXT,
                number_of_likes INT,
                number_of_shares INT,
                FOREIGN KEY(author) REFERENCES twitter_data.Accounts(author),
                FOREIGN KEY(language) REFERENCES twitter_data.language(language)
                )""")
    print(f"Table Tweets created ")


Table Tweets created 


#### Create Foreign key Index

In [169]:
with conn.cursor() as cur:
    cur.execute("""CREATE INDEX idx_tweets_author_fk ON twitter_data.Tweets (author)""")
    cur.execute("""CREATE INDEX idx_tweets_language_fk ON twitter_data.Language (language)""")


### Insert data to Tables:

In [170]:
accounts_insert_query = (""" INSERT INTO twitter_data.Accounts(author) VALUES (%s)""")

accounts_data = df[['author']].drop_duplicates()
accounts_values = [tuple(x) for x in accounts_data.values]
# print(accounts_data.values)
with conn.cursor() as cur:
    cur.executemany(accounts_insert_query,accounts_values)

In [171]:
language_insert_query = (""" INSERT INTO twitter_data.Language(language) VALUES (%s)""")
language_data = df[['language']].drop_duplicates()
language_values = [tuple(x) for x in language_data.values]
with conn.cursor() as cur:
    cur.executemany(language_insert_query,language_values)

In [172]:
tweets_insert_query = (
    """ INSERT INTO twitter_data.Tweets(author,content,tweeted_time,tweet_id,language,number_of_likes,number_of_shares) VALUES (%s,%s,%s,%s,%s,%s,%s)""")

tweets_data = [tuple(x) for x in df.values]
with conn.cursor() as cur:
    cur.executemany(tweets_insert_query,tweets_data)

In [175]:
conn.close()